In [1]:
import pandas as pd
filein = "/Users/buixuanminh/Desktop/LLM model/Data/Qst_CourseEval_Data.csv"
data = pd.read_csv(filein)
data.head(5)

,Semester,Industror,Course,Section,Title,Course Count,Course Mean,Instr Count,Instr Mean,Response Percentage,Year
0,Fall,"Hagigi, Moshe",AC221,A1,Financial Accounting,32.0,3.91,32.0,3.84,97%,2012
1,Spring,"Huang, Wenli",AC221,A1,Financial Accounting,16.0,4.19,16.0,4.44,76%,2012
2,Summer,"Thammasiri, Wasinee",AC221,A1,Financial Accounting,41.0,4.12,41.0,4.12,93%,2012
3,Summer,"Kelly, Paula",AC221,A2,Financial Accounting,43.0,4.09,43.0,4.40,88%,2012
4,Fall,"Malik, Mahfuja",AC221,B1,Financial Accounting,15.0,4.07,15.0,4.00,88%,2012


In [2]:
class DataFile:
    def __init__ (self, data,constant_col,varible_col,sort_col):
        self.data = data
        self.constant_col = constant_col
        self.varible_col = varible_col
        self.sort_col = sort_col

    def show_data(self,limit):
        return self.data.head(limit)
        
    def show_columns(self):
        return list(self.data.columns)

# Trying to get back to OOP (notify to comeback)

In [3]:
# ----------------------- Ultimate Goal of the this code block ----------------------------
# To clean the data set due to changing different course name and course number
# To standardize the name of the number to the most recent (thanks to questrom inconsistent naming)

# This function does the following:
    # 1. Filter out the data to have 2 main columns
    # 2. Grouped data through constant classification/col and counted the number of unique variable classification/col
    # 3. Filter out data with >1 unique variable classification (having a duplicate of a supposed constant classification)
    # 4. Return the list of duplicated constant classification (to force all variables to single constant classification)

# Example:
    # 1. constant: (Math_Level_A), variable: (MA123, AD123, MT125) --> Math_Level_A 
    # 2. constant: (EL123), variable (English1) --> Removed to filter

def duplicated_classification_list (data, constant_col, varible_col):
    filter_data = data[[constant_col,varible_col]]
    distint_count = filter_data.groupby(constant_col).agg("nunique").reset_index()
    Unique_filter = distint_count[varible_col] > 1
    Duplicate_var_classification_table = distint_count[Unique_filter].reset_index(drop=True)
    constant_list_with_duplication = list(Duplicate_var_classification_table[constant_col])
    
    return constant_list_with_duplication

A = duplicated_classification_list(data,"Title","Course")
B = duplicated_classification_list(data,"Course","Title")

# This function does the following:
    # Created a loop to go through every data point from a list of duplicated constant classification
    # 1. From the data filter out the column that has the data point
    # 2. Sorted the filter data point based on year descending to get the latest classification first
    # 3. Remove the duplicated value in the sorted_list using dict
    # 4. Replace data of the variable after the first one (previous name) with the first variable in the list (latest name)
    # Rinse & repeat

def clean_changed_classification (data, constant_list_with_duplication ,constant_col, varible_col, sort_col):
    specific_data = data[[constant_col,varible_col,sort_col]]
    
    for element in constant_list_with_duplication:
        constant_col_filter = specific_data[constant_col] == element
        filted_data = specific_data[constant_col_filter]
        sorted_data = filted_data.sort_values(by = [sort_col], ascending = False).reset_index()
        # error happened here (specific data didn't have column to sort_col initially)
        sorted_list = list(sorted_data[varible_col])
        unique_list_of_varible_col_element = list(dict.fromkeys(sorted_list))

        main_varible = unique_list_of_varible_col_element[0]
        varibles_to_replace = unique_list_of_varible_col_element [1::] 
        
        data[varible_col] = data[varible_col].replace(varibles_to_replace, main_varible)
        # error happened here (variable didn't change) --> rethink the code logic 

    cleaning_check = duplicated_classification_list(data,constant_col,varible_col)
    if len(cleaning_check) == 0:
        return data
    else:
        print("Error uncleaned")

# Just because I am lazy

def combined_function(data,constant_col, varible_col,sort_col):
    Title_constant_list = duplicated_classification_list(data, constant_col, varible_col)
    Cleaned_title_data = clean_changed_classification(data, Title_constant_list, constant_col, varible_col, sort_col)
    
    Course_constant_list = duplicated_classification_list(data, varible_col,constant_col)
    Cleaned_course_data = clean_changed_classification(data, Course_constant_list, varible_col, constant_col, sort_col)

    Final_cleaned_data = Cleaned_course_data
    return Final_cleaned_data


In [4]:
# multiple courses for 1 title: changing course classification
    # Const: Managing people, Var: MO221, OB221 --> Managing people = MO221
# multiple titles for 1 course: changing name
    # Const: FE342, Var: Money&Market, Money and Market --> FE342 = Money&Market

# Due to the inconsistency of questrom in both naming course and numbering course
# The cleaning need to happen twice one with the title as constant and another one course as constant

Cleaned_data = combined_function(data,"Title", "Course","Year")
Cleaned_data.head(2)

,Semester,Industror,Course,Section,Title,Course Count,Course Mean,Instr Count,Instr Mean,Response Percentage,Year
0,Fall,"Hagigi, Moshe",AC221,A1,Financial Accounting,32.0,3.91,32.0,3.84,97%,2012
1,Spring,"Huang, Wenli",AC221,A1,Financial Accounting,16.0,4.19,16.0,4.44,76%,2012


In [5]:
ListA = [1,1,2,1,4,1,9,3,3,1,4]
ListB = [1,3]
Holder = []

def isin_check_list (datapoint, check_list):
    result = ""
    for element in check_list:
        if element == datapoint:
            return True
    return False

def check_isin_list (Original_list, Check_list):
    holder = []
    for element in Original_list:
        In_list_or_not = isin_check_list(element, Check_list)
        holder.append(In_list_or_not)
    return holder

In [19]:
# Additional cleaning by hand part 1
# 1. Drop N/A (questrom haven't input those data)
# 2. Create a filter of active Instructors
# 3. Rename the Instructor column name

Cleaned_data = Cleaned_data.dropna().reset_index(drop = True)

Max_Year_Filter = Cleaned_data.Year == Cleaned_data.Year.max()
Active_instructor_list = list(Cleaned_data[Max_Year_Filter].Industror)
Active_instructor_list = list(dict.fromkeys(Active_instructor_list))
Instructor_list = list(Cleaned_data.Industror)
Active_instructor_filter = check_isin_list(Instructor_list, Active_instructor_list)

# Or I can just use this single line of code -______-
#Active_instructor_filter = Cleaned_data.Industror.isin(Active_instructor_list)

Cleaned_data["Active_Instructor"] = Active_instructor_filter
Cleaned_data_final = Cleaned_data.rename(columns={'Industror': 'Instructor'})

In [7]:
# Additional cleaning by hand part 2
# I am just trying to add a date time column to the data how hard could it be 

Cleaned_data_final = Cleaned_data.rename(columns={'Industror': 'Instructor'})
copied_cleaned = Cleaned_data_final
copied_cleaned.Year = copied_cleaned.Year.astype(str)
copied_cleaned.Semester = copied_cleaned.Semester.str.lower()
Replace_list = ["fall","spring","summer"]
Replace_to_list = ["December/20/","May/18/","August/09/"]

def replacing_list (data, Replace_list,Replace_to_list,column_to_replace): 
    index = 0
    data = data[column_to_replace]
    while index < len(Replace_list):
        data.replace(Replace_list[index], Replace_to_list[index], inplace = True)
        index = index + 1
    return data

Date_dataframe = replacing_list(copied_cleaned,Replace_list,Replace_to_list,"Semester")
Date_dataframe = Date_dataframe + copied_cleaned.Year
Cleaned_data_final["Date_Time"] = Date_dataframe
Cleaned_data_final["Date_Time"] = pd.to_datetime(Cleaned_data_final["Date_Time"])

# God forbid i try to replace data in a copied column and not affected the original one
Cleaned_data_final.Semester = Cleaned_data.Semester

In [8]:
# Additional cleaning by hand #3 (the fate of not having Power BI premium)
# Added the department column

Course_list = list(Cleaned_data_final.Course)
Department_list = [data_point[:2] for data_point in Course_list]
Cleaned_data_final["Department"] = Department_list

Cleaned_data_final

,Semester,Instructor,Course,Section,Title,Course Count,Course Mean,Instr Count,Instr Mean,Response Percentage,Year,Active_Instructor,Date_Time,Department
0,Fall,"Hagigi, Moshe",AC221,A1,Financial Accounting,32.0,3.91,32.0,3.84,97%,2012,True,2012-12-20,AC
1,Spring,"Huang, Wenli",AC221,A1,Financial Accounting,16.0,4.19,16.0,4.44,76%,2012,False,2012-05-18,AC
2,Summer,"Thammasiri, Wasinee",AC221,A1,Financial Accounting,41.0,4.12,41.0,4.12,93%,2012,False,2012-08-09,AC
3,Summer,"Kelly, Paula",AC221,A2,Financial Accounting,43.0,4.09,43.0,4.40,88%,2012,False,2012-08-09,AC
4,Fall,"Malik, Mahfuja",AC221,B1,Financial Accounting,15.0,4.07,15.0,4.00,88%,2012,False,2012-12-20,AC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6392,Spring,"Kariv, Karen",SM275,P1,Management Communications,18.0,3.50,18.0,4.11,82%,2023,True,2023-05-18,SM
6393,Spring,"Wilcke, Marilyn (Midge)",SM275,Q1,Management Communications,16.0,3.44,16.0,4.38,76%,2023,True,2023-05-18,SM
6394,Spring,"Feldman, Pnina",SM456,A1,Management Honors Seminar,35.0,3.97,36.0,4.64,70%,2023,True,2023-05-18,SM
6395,Fall,"Zapatero, Fernando",SM456,A1,Management Honors Seminar,28.0,3.93,28.0,4.25,82%,2023,True,2023-12-20,SM


In [9]:
Cleaned_data_final.to_csv("/Users/buixuanminh/Desktop/Cleaned_QST_Eval.csv")